In [24]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

import gym
from IPython.display import display

from collections import namedtuple
from dataclasses import dataclass, field

Observe the env.

In [3]:
env = gym.make('CartPole-v0')
env.reset()

for i in range(100):
    env.render()
    
    env.step(env.action_space.sample())
    
env.close()

In [6]:
class DQN(nn.Module):
    
    def __init__(self, width, height):
        super().__init__()
        
        self.fc1 = nn.Linear(in_features=widht*height*3, out_features=24)
        self.fc2 = nn.Linear(in_features=24, out_features=32)
        self.out = nn.Linear(in_features=32, out_features=2)
        
    def forward(self, t):
        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        return t

In [18]:
Experiance = namedtuple(
    "Experiances",
    ("action", "state", "next_state", "reward")
)

In [22]:
Exception(1, 2, 3, 4)

Exception(1, 2, 3, 4)

In [39]:
@dataclass(init=True, repr=False)
class ReplayMemory:
    
    """
        ReplayMemory class is for handling the replay memory data and sample batch_size samples for training.
    """
    
    capacity: int
    memory: list = field(default_factory=list)
    push_count: int = 0
        
    def push(self, exp):
        if len(self.memory) < self.capacity:
            self.memory.append(exp)
        else:
            self.memory[sefl.push_count % capacity] = exp
        self.push_count += 1
        
    def sample(self, batch_size):
        return np.random.choice(self.memory, batch_size, replace=False)
    
    def can_provide_sample(self):
        return len(self.memory) >= batch_size

In [72]:
@dataclass(init=True)
class EpGreedy:
    
    start: int
    end: int
    decay: int
        
    def get_exploration_rate(self, current_step):
#         if current_step - self.decay >= self.end:
#             return current_step - self.decay
#         return current_step
        return self.end + (self.start - self.end) * np.exp(-1 * current_step * self.decay)

In [73]:
ep_greedy = EpGreedy(1, 0.1, 0.1)

In [76]:
ep_greedy.get_exploration_rate(0.2)

0.9821788059760798

**RL Agent**

In [77]:
class Agent:
    
    def __init__(self, strategy, num_actions):
        self.strategy = strategy
        self.num_actions = num_actions
        self.current_step = 0
        
    def select_action(self, state, policy_net):
        rate = self.strategy.get_exploration_rate(current_step)
        self.current_step += 1
        
        if rate > random.random():
            return np.random.choice(self.num_actions) # explore
        else:
            with torch.no_grad():
                return policy_net(state).argmax(dim=1).item() # exploit